# Chatbot + Recomanació

L'objectiu d'aquesta pràctica és força directe, volem recomanar a l'usuari noves `aisle_id` on anar a comprar quan ens ho demani.

Es dóna tant un chatbot funcional com un recomanador basat en factorització de matrius, però si voleu podeu fer servir les vostres implementacions pròpies del chatbot i del recomanador basat en pagerank.

Si feu servir el codi donat, llegiu les consideracions que trobareu més abaix

In [1]:
import zipfile
from os.path import join, dirname
import pandas as pd

def locate(*path):
    base = globals().get('__file__', '.')
    return join(dirname(base), *path)

def unzip(file):
    zip_ref = zipfile.ZipFile(locate(file), 'r')
    zip_ref.extractall(locate('data'))
    zip_ref.close()

unzip('order_products__train.csv.zip')
unzip('orders.csv.zip')
unzip('products.csv.zip')
unzip('aisles.csv.zip')

In [2]:
df_order_prods = pd.read_csv(locate('data', 'order_products__train.csv'))
df_orders = pd.read_csv(locate('data', 'orders.csv'))[['order_id', 'user_id']]
df_prods = pd.read_csv(locate('data', 'products.csv'))[['product_id', 'aisle_id']]

df_merged = pd.merge(pd.merge(df_order_prods, df_orders, on='order_id'), df_prods, on='product_id')
counts = df_merged.groupby(['user_id', 'aisle_id']).size()
df_counts = counts.unstack()

In [3]:
df_aisles = pd.read_csv(locate('data', 'aisles.csv'), index_col='aisle_id')

In [4]:
if __name__ == '__main__':
    print(df_aisles.head())

                               aisle
aisle_id                            
1              prepared soups salads
2                  specialty cheeses
3                energy granola bars
4                      instant foods
5         marinades meat preparation


## Pràctica

**Punt 1.** L'usuari solament ha de poder afegir productes que es trobin en el dataframe `df_aisles`, és a dir, el nom del producte s'ha de trobar en la columna `aisles` d'aquest dataframe

**Punt 2.** Quan l'usuari enviï la comanda `/reco` i solament si es troba afegint productes (podeu cridar `user.is_adding()` per comprovar-ho), li heu de recomanar un nou producte que no tingui ja a la llista. Per fer-ho, els passos que seguirem seran els següents:

1. Buscar a la base de dades (`df_counts`) la persona més semblant a l'usuari del bot. Això es pot fer de diverses maneres, per exemple, pots mirar quina persona té una distància més petita respecte l'usuari tenint en compte les compres, amb `np.linalg.norm(compres_persona_db - llista_usuari)`, o agant la que té la correlació de pearson més gran entre les seves compres i la de l'usuari (`scipy.stats.stats.pearsonr` o el mètode `corr` dels dataframes).

Està clar, per fer això necessites la llista de productes afegits de l'usuari en funció de `aisle_id` (no el nostre `product_id`) i de la quantitat `qty`, pots obtenir-ho a partir de `user_info(user.id)['products']`.

2. Un cop tens aquesta persona, calcula el seu `score` (ie. l'estimació de compra) per totes les `aisle_id` que l'usuari no hagi comprat encara.

3. Envia un missatge a l'usuari amb el nom de la `aisle_id` que ha tret millor puntuació en el punt anterior i la puntuació arrodonida a l'enter més proper

## Consideracions del recomanador

El recomanador es dóna ja entrenat (arxius P.pkl i Q.pkl), però de forma ràpida i poc fiable. Podeu obtenir la recomanació d'un usuari de la base de dades (df_counts) per a un item (aisle_id) donat amb el mètode `estimate(usuari, item)`.

In [42]:
from recommender import NMFRecommender

reco = NMFRecommender(df_counts, 3, 10)
reco.factorize()
reco.estimate(1, 2)
print(df_aisles.shape)
k = ["instant foods", "coffee", "soap"]
k2 = df_aisles[df_aisles["aisle"].isin(k)].index.tolist()
kk = np.zeros(df_aisles.shape[0])
for i in k2:
    kk[i] += 1
    
elnuevok = df_counts.fillna(0)

print(elnuevok - kk)


(134, 1)
aisle_id  1    2    3    4    5    6    7    8    9    10  ...   125  126  \
user_id                                                    ...              
1         0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
2         1.0  0.0  1.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
5         0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
7         0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
8         0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
9         0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
10        0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
13        0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
14        0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
17        0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
18        0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0 ...   0.

## Consideracions del chatbot

* No teniu accés directe al codi del bot, treballareu a partir de funcions "callback", és a dir, quan el bot detecta un event cridarà les vostres funcions. Les funcions, totes elles, tenen un dos paràmetres en comú:
    * `bot`: Objecte DelegatorBot de Telepot
    * `user`: Objecte ShoppingUser
    
Les funcions que es cridaran a mode de callback són:

* Quan es rep una comanda, és a dir un missatge que comença per /, es cridarà `on_cmd(bot, user, cmd)`. El paràmetre `cmd` conté la comanda enviada per l'usuari
* Quan s'afegeix un producte, es crida `on_add(bot, user, product_id, qty)`, on `product_id` indica el nom del producte i `qty` la quantitat comprada. Si aquesta funció retorna True o None, l'item s'afegirà a l'usuari, però si retorna False **no** s'afegirà.
* Quan es marca un producte com a comprat (si encara no estava comprat), es crida `on_flag(bot, user, product_id)`
* Quan s'acaben de comprar tots els productes, i solament 1 cop per interacció, es crida `on_end(bot, user)`

**Els productes de l'usuari ja no són una llista de productes, sinó diccionari de productes**:

```python
{
    ...
    'products': {
        'product_id_1': {
            'status': 0/1,
            'qty': <int>
        },
        ...
        'product_id_n': {
            'status': 0/1,
            'qty': <int>
        },
    }
    ...
}
```

In [ ]:
import numpy as np

async def on_cmd(bot, user, cmd):
    if cmd == '/start':
        await bot.sendPhoto(user.id, open('img/hello.jpg', 'rb'))
    elif cmd == '/done':
        await user.sender.sendMessage('Let\'s go')
    elif cmd == '/reco':
        
        
    
    print(bot)
    print(user)
    print(user_info(user.id))
    
async def on_add(bot, user, product_id, qty):
    #add_product(user.id, product_id, qty)
    print(product_id)
    if (df_aisles[df_aisles == product_id].count()[0]==1):
        return True
    else:
        return False
        
async def on_flag(bot, user, product_id):
    pass
    
async def on_end(bot, user):
    await bot.sendPhoto(user.id, open('img/done.png', 'rb'))
    pass

In [12]:
if __name__ == '__main__':
    from bot import ShoppingBot, user_info
    
    with ShoppingBot() as bot:
        # Setup callbacks
        bot.add_callback('cmd', on_cmd)
        bot.add_callback('add-product', on_add)
        bot.add_callback('flag-product', on_flag)
        bot.add_callback('end', on_end)
        
        # Start bot
        bot.start(open('TOKEN').read().strip())

{'id': 449314022, 'status': 0, 'messages': [], 'products': {}}
specialty cheese


Traceback (most recent call last):
  File "/home/ziclon/anaconda/envs/py36/lib/python3.6/site-packages/telepot/aio/delegate.py", line 92, in wait_loop
    await helper._invoke(j.on_message, msg)
  File "/home/ziclon/anaconda/envs/py36/lib/python3.6/site-packages/telepot/aio/helper.py", line 16, in _invoke
    return await fn(*args, **kwargs)
  File "/home/ziclon/anaconda/envs/py36/lib/python3.6/site-packages/telepot/aio/helper.py", line 191, in augmented
    return await _invoke(handler, msg)
  File "/home/ziclon/anaconda/envs/py36/lib/python3.6/site-packages/telepot/aio/helper.py", line 16, in _invoke
    return await fn(*args, **kwargs)
  File "/home/ziclon/anaconda/envs/py36/lib/python3.6/site-packages/telepot/aio/helper.py", line 269, in on_message
    await self._router.route(msg)
  File "/home/ziclon/anaconda/envs/py36/lib/python3.6/site-packages/telepot/aio/helper.py", line 244, in route
    return await _invoke(fn, msg, *args, **kwargs)
  File "/home/ziclon/anaconda/envs/py36/l

KeyboardInterrupt: 